In [1]:
!pip install scikit-learn

In [2]:
import zipfile
# from google.colab import files
import os

import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import keras
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

In [3]:
labels = np.load('labels.npy', mmap_mode = 'r')
snrs = np.load('snrs.npy', mmap_mode = 'r')
signals = np.load('signals.npy', mmap_mode = 'r')

In [4]:
part = 3
signals = signals[::part, :, :]
labels = labels[::part, :]
snrs = snrs[::part, :]

print(signals.shape)
print(labels.shape)
print(snrs.shape)

(851968, 1024, 2)
(851968, 24)
(851968, 1)


In [5]:
snrs = np.ravel(snrs)
print(f"All possible SNRS: {np.unique(snrs)} db")

All possible SNRS: [-20. -18. -16. -14. -12. -10.  -8.  -6.  -4.  -2.   0.   2.   4.   6.
   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.  28.  30.] db


In [6]:
c = np.ma.masked_where(snrs > 8, snrs)
msk = c.mask
# Count unique elements in array
print(np.unique(c.mask, return_counts=True))

(array([False,  True]), array([491520, 360448], dtype=int64))


In [7]:
signals = signals[msk]
labels = labels[msk]

print(len(signals))

360448


In [8]:
# Train|test = 80|20

x_train,x_test, y_train, y_test = train_test_split(signals, labels, train_size=0.8, stratify=labels)

# print(f"Количество строк в y_train по классам: {np.bincount(y_train)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_test)}")
print(x_test.shape)
print(y_test.shape)

# Train|validation|test = 64|16|20
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, stratify=y_train)

# print(f"Количество строк в y_train по классам: {np.bincount(y_test)}")
# print(f"Количество строк в y_test по классам: {np.bincount(y_val)}")
print(x_train.shape)
print(y_train.shape)

(72090, 1024, 2)
(72090, 24)
(230686, 1024, 2)
(230686, 24)


In [9]:
class Residual_block:
    kernel_size = 3
    strides = 1
    padding = 'same'
    data_format = "channels_last"

    def __init__(self, x, x_shortcut, filters):
        self.x = x
        self.filters = filters
        self.x_shortcut = x_shortcut

    def unit(self):
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(self.x)
        x = Activation('relu')(x)
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(x)
        x = Activation('linear')(x)
        # add skip connection
        if x.shape[1:] == self.x_shortcut.shape[1:]:
            x = Add()([x, self.x_shortcut])
        else:
            raise Exception('Skip Connection Failure!')
        return x

In [10]:
class Convolution_block:
    kernel_size = 1
    strides = 1
    padding = 'same'
    data_format = "channels_last"

    def __init__(self, x, filters):
        self.x = x
        self.filters = filters

    def unit(self):
        x = Conv1D(self.filters, self.kernel_size, self.strides, self.padding, self.data_format)(self.x)
        x = Activation('linear')(x)
        return x

In [11]:
def residual_stack(x, filters):
    x = Convolution_block(x, filters)
    print('x')
#     print(x.shape)
    print(x)
    x = x.unit()
    print('xunit')
#     print(x.shape)
    print(x)
    
    x_shortcut = x
    x = Residual_block(x, x_shortcut, filters)
    x = x.unit()
    x_shortcut = x
    x = Residual_block(x, x_shortcut, filters)  
    x = x.unit()
    
    # max pooling layer
    x = MaxPooling1D(pool_size=2, strides=None, padding='valid', data_format='channels_last')(x)
#     print('Residual stack created')
    return x

In [12]:
def ResNet(input_shape, classes):   
    # create input tensor
    x_input = Input(input_shape)
    x = x_input
    # residual stack
    num_filters = 40
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    x = residual_stack(x, num_filters)
    
    # output layer
    x = Flatten()(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(128, activation="selu", kernel_initializer="he_normal")(x)
    x = Dropout(.5)(x)
    x = Dense(classes , activation='softmax', kernel_initializer = glorot_uniform(seed=0))(x)
    
    # Create model
    model = Model(inputs = x_input, outputs = x)
#     print('Model ResNet created')
    return model

In [13]:
save_model = False
save_history = False

# create directory for model weights
if save_model is True:
    weights_path = input("Name model weights directory: ")
    weights_path = "data/weights/" + weights_path

    try:
        os.mkdir(weights_path)
    except OSError:
        print ("Creation of the directory %s failed" % weights_path)
    else:
        print ("Successfully created the directory %s " % weights_path)
    print('\n')
    

# create directory for model history
if save_history is True:
    history_path = input("Name model history directory: ")
    history_path = "data/model_history/" + history_path

    try:
        os.mkdir(history_path)
    except OSError:
        print ("Creation of the directory %s failed" % history_path)
    else:
        print ("Successfully created the directory %s " % history_path)
    print('\n')

In [14]:
adm = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

# set number of epochs
num_epochs = int(input('Enter number of epochs: '))

# set batch size
batch = 32

# configure weights save

if save_model is True:
    filepath= weights_path + "/{epoch}.hdf5"
    checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=False, mode="auto")
    callbacks_list = [checkpoint]
else:
    callbacks_list = []

Enter number of epochs: 50


In [15]:
model = ResNet((1024, 2), 24)
model.compile(optimizer=adm, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
history = model.fit(x_train, y_train, epochs = num_epochs, batch_size = batch, callbacks=callbacks_list, validation_data=(x_val, y_val))

x
xunit
KerasTensor(type_spec=TensorSpec(shape=(None, 1024, 40), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'activation'")
x
xunit
KerasTensor(type_spec=TensorSpec(shape=(None, 512, 40), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'activation_5'")
x
xunit
KerasTensor(type_spec=TensorSpec(shape=(None, 256, 40), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'activation_10'")
x
xunit
KerasTensor(type_spec=TensorSpec(shape=(None, 128, 40), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'activation_15'")
x
xunit
KerasTensor(type_spec=TensorSpec(shape=(None, 64, 40), dtype=tf.float32, name=None), name='Placeholder:0', description="created by layer 'activation_20'")
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Conne